In [127]:
from gurobipy import *
import pandas as pd
import numpy as np
import grblogtools as glt
from scipy.spatial.distance import squareform, pdist
import datetime
import math

In [128]:
df = pd.read_csv('data\instance03.txt', header=None)
n_S = int(df_main['n_S'])
n_C = int(df_main['n_C'])
n_L = int(df_main['n_L'])
n_K = int(df_main['n_K'])
n_D = int(df_main['n_D'])
B = int(df_main['B'])//30

df_main = df.iloc[:2,:]
df_main.columns = df_main.iloc[0]
df_main = df_main[1:]

df_car =  df.iloc[3:n_C+4,:]
df_car.columns = df_car.iloc[0]
df_car = df_car.dropna(axis=1)
df_car = df_car[1:]
print(df_car)

df_car_lv = df.iloc[n_C+5:n_C+n_L+6,:]
df_car_lv.columns = df_car_lv.iloc[0]
df_car_lv = df_car_lv.dropna(axis=1)
df_car_lv = df_car_lv[1:]
print(df_car_lv)

df_order = df.iloc[n_C+n_L+7:n_C+n_L+n_K+8,:]
df_order.columns = df_order.iloc[0]
df_order = df_order.dropna(axis=1)
df_order = df_order[1:]
df_order['Time span'] = df_order.apply(lambda row: pd.to_datetime(row['Return time']) - pd.to_datetime(row['Pick-up time']), axis=1)
df_order['Time units'] = df_order['Time span'].apply(lambda x: math.ceil(x.total_seconds() / 1800))
first_period_start = pd.to_datetime('2023/01/01 00:00')
time_unit = pd.Timedelta('30 minutes')
df_order['Pick-up time (ordinal)'] = df_order.apply(lambda row: pd.to_datetime(row['Pick-up time']) - first_period_start, axis=1).apply(lambda x: math.ceil(x.total_seconds() / 1800))
df_order['Return time (ordinal)'] = df_order.apply(lambda row: pd.to_datetime(row['Return time']) - first_period_start, axis=1).apply(lambda x: math.ceil(x.total_seconds() / 1800))
print(df_order)

df_distance = df.iloc[n_C+n_L+n_K+9:n_C+n_L+n_K+n_S**2+10,:]
df_distance.columns = df_distance.iloc[0]
df_distance = df_distance.dropna(axis=1)
df_distance = df_distance[1:]
distance = df_distance.pivot(index='From', columns='To', values='Distance')
distance = distance.apply(pd.to_numeric, errors='coerce') // 30

print(distance)

3 Car ID Level Initial station
4      1     1               1
5      2     1               2
6      3     1               1
7      4     1               3
8      5     1               2
10 Car level Hour rate
11         1       100
13 Order ID Level Pick-up station Return station      Pick-up time  \
14        1     1               2              2  2023/01/01 06:00   
15        2     1               3              3  2023/01/01 00:30   
16        3     1               1              1  2023/01/02 20:30   
17        4     1               3              2  2023/01/05 13:00   
18        5     1               2              1  2023/01/02 09:30   
19        6     1               2              3  2023/01/03 03:00   
20        7     1               2              1  2023/01/06 06:00   
21        8     1               3              1  2023/01/06 16:00   
22        9     1               1              3  2023/01/04 16:30   
23       10     1               1              3  2023/01/08 01:00  

In [109]:
eg1 = Model("eg1")

#-------- Add variables as a list ---------#
# xmk = 1 if order k is processed on car m
x = []
for m in range(n_C):
    x.append([])
    for k in range(n_K):
        x[m].append(eg1.addVar(lb = 0, vtype = GRB.BINARY, name = "x" + str(m+1) + str(k+1)))

# Tij = rearrangment time to move car from station i to station j
T = []
for i in range(n_S):
    T.append([])
    for j in range(n_S):
        T[i].append(eg1.addVar(lb=distance.iloc[j, i], vtype=GRB.INTEGER, name="T" + str(i+1) + str(j+1)))

# Ck = completion time of order k
C = []
for k in range(n_K):
    C.append(eg1.addVar(lb=0, vtype = GRB.INTEGER, name = "C" + str(k+1)))

# Om = latest completion time of orders on car m
O = []
for m in range(n_C):
    O.append(eg1.addVar(lb=0, vtype = GRB.INTEGER, name = "O" + str(m+1)))



In [ ]:
eg1.setObjective(
    quicksum(R[k] for k in n_K)\
    ,GRB.MAXIMIZE
)